# Synchronous Generator - Classic Transient Stability Model

In [ ]:
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import dpsimpy
import numpy as np

## Steady State

In [ ]:
# FIXME: Why does this have to run in a seperate function for the simulation to produce the Left- and RightVector log files?
def DpSynGenTrStab_SteadyState():
    # Define simulation parameters
    time_step = 0.0005
    final_time = 0.03
    sim_name = 'DP_SynGen_TrStab_SteadyState'
    dpsimpy.Logger.set_log_dir('logs/' + sim_name)

    # Define machine parameters in per unit
    nom_power = 555e6
    nom_ph_ph_volt_rms = 24e3
    nom_freq = 60
    H = 3.7
    Ll = 0.15
    Lmd = 1.6599
    Llfd = 0.1648

    # Initialization parameters
    init_elec_power = complex(300e6, 0)
    init_terminal_volt = 24000
    init_volt_angle = 0
    init_voltage = init_terminal_volt * np.exp(init_volt_angle * 1j)
    mech_power = 300e6

    # Define grid parameters
    R_load = 1.92

    # Nodes
    n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.Single, [init_voltage])

    # Components
    gen = dpsimpy.dp.ph1.SynchronGeneratorTrStab('SynGen', dpsimpy.LogLevel.debug)
    gen.set_fundamental_parameters_PU(nom_power, nom_ph_ph_volt_rms, nom_freq, Ll, Lmd, Llfd, H)
    gen.connect([n1])
    gen.set_initial_values(init_elec_power, mech_power)

    res = dpsimpy.dp.ph1.Resistor('Rl', dpsimpy.LogLevel.debug)
    res.set_parameters(R_load)
    res.connect([dpsimpy.dp.SimNode.gnd, n1])

    # System
    sys = dpsimpy.SystemTopology(60, [n1], [gen, res])

    # Logging
    logger = dpsimpy.Logger(sim_name)
    logger.log_attribute('v1', 'v', n1)
    logger.log_attribute('i_gen', 'i_intf', gen)
    logger.log_attribute('i_load', 'i_intf', res)
    logger.log_attribute('wr_gen', 'w_r', gen)

    # Simulation
    sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.trace)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_system(sys)
    sim.set_time_step(time_step)
    sim.set_final_time(final_time)
    sim.set_domain(dpsimpy.Domain.DP)
    sim.add_logger(logger)
    sim.run()


In [ ]:
DpSynGenTrStab_SteadyState()

In [ ]:
# read log file
work_dir = 'logs/DP_SynGen_TrStab_SteadyState/'
logName_ststate = 'DP_SynGen_TrStab_SteadyState_LeftVector'
print(work_dir + logName_ststate + '.csv')
ts_dpsim_ststate = rt.read_timeseries_dpsim(work_dir + logName_ststate + '.csv')
ts_dpsim_ststate_emt = ts.frequency_shift_list(ts_dpsim_ststate, 60)
phasors_ststate = ts.phasors(ts_dpsim_ststate)

In [ ]:
for node, phasor in phasors_ststate.items():
    print(node + ': ' + str(phasor['abs'].values[0]) + '<' + str(phasor['phase'].values[0]))

In [ ]:
pt.plot_timeseries(1, ts_dpsim_ststate_emt['n00000f00_shift'])
pt.plot_timeseries(1, ts_dpsim_ststate_emt['n00001f00_shift'])

In [ ]:
# read log file
work_dir = './logs/DP_SynGen_TrStab_SteadyState/'
logName_ststate = 'DP_SynGen_TrStab_SteadyState'
print(work_dir + logName_ststate + '.csv')
ts_dpsim_ststate = rt.read_timeseries_dpsim(work_dir + logName_ststate + '.csv')
ts_dpsim_ststate_emt = ts.frequency_shift_list(ts_dpsim_ststate, 60)
phasors_ststate = ts.phasors(ts_dpsim_ststate)

In [ ]:
print('v1' + ': ' + str(phasors_ststate['v1']['abs'].values[0]) + '<' + str(phasors_ststate['v1']['phase'].values[0]))

In [ ]:
pt.plot_timeseries(1, ts_dpsim_ststate['wr_gen'])
plt.ylim([375,380])

In [ ]:
import numpy as np
assert np.max(ts_dpsim_ststate['wr_gen'].values[0] - ts_dpsim_ststate['wr_gen'].values[-1]) < 0.001

## Load Step

In [ ]:
# Define simulation parameters
time_step = 0.0005
final_time = 0.1
sim_name = 'DP_SynGen_TrStab_LoadStep'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
H = 3.7
Ll = 0.15
Lmd = 1.6599
Llfd = 0.1648

# Initialization parameters
init_elec_power = complex(300e6, 0)
init_terminal_volt = 24000
init_volt_angle = 0
init_voltage = init_terminal_volt * np.exp(init_volt_angle * 1j)
mech_power = 300e6

# Define grid parameters
R_load = 1.92
R_load_step = 0.7

# Nodes
n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.Single, [init_voltage])

# Components
gen = dpsimpy.dp.ph1.SynchronGeneratorTrStab('SynGen', dpsimpy.LogLevel.debug)
gen.set_fundamental_parameters_PU(nom_power, nom_ph_ph_volt_rms, nom_freq, Ll, Lmd, Llfd, H)
gen.connect([n1])
gen.set_initial_values(init_elec_power, mech_power)

load = dpsimpy.dp.ph1.Switch('StepLoad', dpsimpy.LogLevel.debug)
load.set_parameters(R_load, R_load_step)
load.connect([dpsimpy.dp.SimNode.gnd, n1])
load.open()

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, load])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_gen', 'i_intf', gen)
logger.log_attribute('i_load', 'i_intf', load)
logger.log_attribute('wr_gen', 'w_r', gen)

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.05, load, True)
sim.add_event(sw1)

sim.run()


In [ ]:
# read log file
work_dir = 'logs/DP_SynGen_TrStab_LoadStep/'
logName = 'DP_SynGen_TrStab_LoadStep'
print(work_dir + logName + '.csv')
ts_dpsim_loadstep= rt.read_timeseries_dpsim(work_dir + logName + '.csv')
ts_dpsim_loadstep_emt = ts.frequency_shift_list(ts_dpsim_loadstep, 60)
phasors_loadstep = ts.phasors(ts_dpsim_loadstep)

In [ ]:
print('v1' + ': ' + str(phasors_loadstep['v1']['abs'].values[0]) + '<' + str(phasors_loadstep['v1']['phase'].values[0]))

In [ ]:
pt.plot_timeseries(2, ts_dpsim_loadstep_emt['v1_shift'])
plt.xlim(0.03, 0.07)  

In [ ]:
pt.plot_timeseries(3, ts_dpsim_loadstep['wr_gen'])

In [ ]:
assert np.max(ts_dpsim_loadstep['wr_gen'].values[0] - ts_dpsim_loadstep['wr_gen'].values[-1]) < 5